In [1]:
import datasets 

from datasets import load_dataset

datasets = load_dataset('json',data_files='simplified-nq-train.jsonl')

/home/eduardo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(datasets)


DatasetDict({
    train: Dataset({
        features: ['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'],
        num_rows: 307373
    })
})


In [3]:
pip install transformers tensorRT

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

2023-10-28 19:58:05.386937: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-28 19:58:05.605520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 19:58:06.375646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
pip install sacremoses

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 5.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=975da7d8c58b2130dd0f3a287a0ffac648225bcc2f7f1a011633b5d234abdde9
  Stored in directory: /home/eduardo/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from transformers import pipeline
import pandas as pd
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")

In [108]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
  
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-en-pt-t5")

model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-en-pt-t5")

enpt_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [110]:
print(enpt_pipeline("translate English to Portuguese: He has been to Hawaii several times"))
print(pipe(">>por<< Tom tried to stab me."))


[{'generated_text': 'Foi várias vezes a Hawaii'}]
[{'translation_text': 'O Tom tentou esfaquear-me.'}]


In [121]:
def extrair_texto_da_lista(lista):
    if isinstance(lista, list):
        textos = []
        for elemento in lista:
            texto = elemento.get('generated_text', '')
            textos.append(texto)
        return textos
    return None


extrair_texto_da_lista(enpt_pipeline("translate English to Portuguese: He has been to Hawaii several times"))

['Foi várias vezes a Hawaii']

In [117]:
sample = datasets['train'][1]

In [14]:
small_set = datasets['train']

In [124]:
df = pd.DataFrame(columns = ['title', 'pergunta', 'answer'])

In [125]:
for i in range(1):
    doc = small_set[i]['document_text']
    init = small_set[i]['annotations'][0]['long_answer']['start_token']
    end = small_set[i]['annotations'][0]['long_answer']['end_token']
    question = small_set[i]['question_text']
    answer = " ".join(doc.split(" ")[init:end])
    answer= enpt_pipeline('translate English to Portuguese: ' + answer)
    print(answer)
    answer = extrair_texto_da_lista(answer)
    print(answer)
    #question = extrair_texto_da_lista(pipe('>>por<<' + question))
    new_row = {'title': [doc], 'pergunta': [question], 'answer': [answer]}
    df_aux = pd.DataFrame(new_row)
    df = pd.concat([df, df_aux])
df

[{'generated_text': '<P > Um exemplo comum de marketing de licença é uma notícia enviada a clientes de'}]
['<P > Um exemplo comum de marketing de licença é uma notícia enviada a clientes de']


,title,pergunta,answer
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...,[<P > Um exemplo comum de marketing de licença...


In [100]:
df

,title,pergunta,answer
0,Email marketing - Wikipedia <H1> Email marketi...,[{'translation_text': 'que é o uso mais comum ...,[ Um exemplo comum de marketing de permissão é...
0,The Mother ( How I Met Your Mother ) - wikiped...,[{'translation_text': 'como i.met sua mãe que ...,"[> Tracy McConnell , mais conhecido como The ..."
0,Human fertilization - wikipedia <H1> Human fer...,[{'translation_text': 'Que tipo de fertilizaçã...,[ O processo de fertilização envolve uma fusão...
0,List of National Football League career quarte...,[{'translation_text': 'quem teve mais vitórias...,[ O quarterback ativo Tom Brady detém os recor...
0,Roanoke Colony - wikipedia <H1> Roanoke Colony...,[{'translation_text': 'o que aconteceu com o a...,[]
